In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn import cross_validation
from sklearn.metrics import classification_report, confusion_matrix

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# run environment

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction = 0.4)
sess_config = tf.ConfigProto(gpu_options=gpu_options)

# load data

In [3]:
def one_hot(y):
    y = y.reshape(len(y))
    n_values = np.max(y) + 1
    return np.eye(n_values)[np.array(y, dtype=np.int32)]  # Returns FLOATS


def load_X(X_path):
    X_list = []
    file = open(X_path, 'r')
    # Read dataset from disk, dealing with text files' syntax
    X_signal = [np.array(item, dtype=np.float32) for item in [
               line.strip().split('\t') for line in file]]
    X_list.append(X_signal)
    file.close()
    return np.transpose(np.array(X_list), (1, 2, 0))


# Load "y" (the neural network's training and testing outputs)
def load_y(y_path):
    file = open(y_path, 'r')
    # Read dataset from disk, dealing with text file's syntax
    y_ = np.array([elem for elem in [line.strip().split('\t') for line in file]], 
                  dtype=np.int32)
    file.close()
    # Substract 1 to each output class for friendly 0-based indexing
    return one_hot(y_)

labels = {'0':'file', '1':'network', '2':'service', '3':'database', '4':'communication', '5':'memory', '6':'driver', 
    '7':'system', '8':'application', '9':'io', '10':'others', '11':'security', '12':'disk', '13':'processor'}

dataset_path = "data_msg_type/"
X_path = dataset_path + "x.txt"
y_path = dataset_path + "y.txt"

X = load_X(X_path)
y = load_y(y_path)
x = X.reshape(len(X), 1, 7, 2)

# Separate our training data into test and training.
print("Separating data into 80% training set & 20% test set...")
train_x, test_x, train_y, test_y = cross_validation.train_test_split(
    x, y, test_size=0.2, random_state=33)#add random state here...
print("Dataset separated.\n")
print(train_x.shape, train_y.shape, test_y.shape)

Separating data into 80% training set & 20% test set...
Dataset separated.

((160000, 1, 7, 2), (160000, 14), (40000, 14))


# define parameters

In [4]:
input_height = 1
input_width = 7
num_channels = 2
num_labels = 14

batch_size = 100
kernel_size = 4
depth = 35
num_hidden = 256

learning_rate = 0.0001
# iteration times
training_epochs = 100
lstm_size = 128

total_batches = train_x.shape[0] // batch_size

n_classes = 14
n_hidden = 128
n_inputs = 10

rnnW = {
    'hidden': tf.Variable(tf.random_normal([n_inputs, n_hidden])),
    'output': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
rnnBiases = {
    'hidden': tf.Variable(tf.random_normal([n_hidden], mean=1.0)),
    'output': tf.Variable(tf.random_normal([n_classes]))
}

# netowrk config

In [5]:
# init nn parameters
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.0, shape = shape)
    return tf.Variable(initial)

# conv
def depthwise_conv2d(x, W):
    return tf.nn.depthwise_conv2d(x,W, [1, 1, 1, 1], padding='VALID')

def apply_depthwise_conv(x,kernel_size,num_channels,depth):
    weights = weight_variable([1, kernel_size, num_channels, depth])
    biases = bias_variable([depth * num_channels])
    return tf.nn.relu(tf.add(depthwise_conv2d(x, weights),biases))
    
def apply_max_pool(x,kernel_size,stride_size):
    return tf.nn.max_pool(x, ksize=[1, 1, kernel_size, 1], 
                          strides=[1, 1, stride_size, 1], padding='VALID')

# build network

In [6]:
X = tf.placeholder(tf.float32, shape=[None,input_height,input_width,num_channels])
Y = tf.placeholder(tf.float32, shape=[None,num_labels])

c = apply_depthwise_conv(X,kernel_size,num_channels,depth)
print(c.shape)
p = apply_max_pool(c,2,1)
c2 = apply_depthwise_conv(p,3,depth*num_channels,depth//10)
print(c2.shape)
c2Reshape = tf.reshape(c2, [-1, 7, n_inputs])
shuff = tf.transpose(c2Reshape, [1, 0, 2])
shuff = tf.reshape(shuff, [-1, n_inputs])

# Linear activation, reshaping inputs to the LSTM's number of hidden:
hidden = tf.nn.relu(tf.matmul(shuff, rnnW['hidden']) + rnnBiases['hidden'])
print(hidden.shape)

# Split the series because the rnn cell needs time_steps features, each of shape:
hidden = tf.split(axis=0, num_or_size_splits=21,value=hidden)

lstm_cell_1 = tf.contrib.rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, 
                                          state_is_tuple=True)
lstm_cell_2 = tf.contrib.rnn.BasicLSTMCell(lstm_size, forget_bias=1.0, 
                                          state_is_tuple=True)

# Stack two LSTM layers, both layers has the same shape
lstm_layers = tf.contrib.rnn.MultiRNNCell([lstm_cell_1, lstm_cell_2], 
                                          state_is_tuple=True)

lstmOutputs, _ = tf.contrib.rnn.static_rnn(lstm_layers, hidden, dtype=tf.float32)
lstmLastOutput = lstmOutputs[-1]
y_ = tf.matmul(lstmLastOutput, rnnW['output']) + rnnBiases['output']
print(y_.shape)

(?, 1, 4, 70)
(?, 1, 1, 210)
(?, 128)
(?, 14)


# train and test

In [ ]:
with tf.name_scope("cost"):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=Y))
    # optimizer = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=
                                                  learning_rate).minimize(loss)

    # Add scalar summary for cost
    tf.summary.scalar("loss", loss)

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(y_, 1)) 
    # Count correct predictions
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float")) 
    # Cast boolean to float to average
    # Add scalar summary for accuracy
    tf.summary.scalar("accuracy", accuracy)


print("net work done")

cost_history = np.empty(shape=[1],dtype=float)

with tf.Session(config=sess_config) as session:
    # create a log writer. run 'tensorboard --logdir=./logs/nn_logs'
    writer = tf.summary.FileWriter("data/cnn_lstm_logs", session.graph)  # for 1.0
    merged = tf.summary.merge_all()

    tf.global_variables_initializer().run()
    for epoch in range(training_epochs):
        for b in range(total_batches):    
            offset = (b * batch_size) % (train_y.shape[0] - batch_size)
            batch_x = train_x[offset:(offset + batch_size), :, :, :]
            batch_y = train_y[offset:(offset + batch_size), :]
            _, c = session.run([optimizer, loss],feed_dict={X: batch_x, Y : batch_y})
            cost_history = np.append(cost_history,c)

            test_indices = np.arange(len(test_x))  # Get A Test Batch
            np.random.shuffle(test_indices)
            test_indices = test_indices[0:]
            summary, acc=session.run([merged, accuracy], 
                        feed_dict={X: test_x[test_indices], Y: test_y[test_indices]})
        print "Epoch: ",epoch," Training Loss: ",c," Training Accuracy: ", acc
        writer.add_summary(summary, epoch)  # Write summary

    print "Testing Accuracy:", session.run(accuracy, feed_dict={X: test_x, Y: test_y})
    writer.close()

net work done


# confusion matrix

In [ ]:
def plot_confusion_matrix(cm, title='Normalized Confusion matrix', 
                          cmap=plt.cm.get_cmap("Blues")): 
    cm = cm / cm.astype(np.float).sum(axis=1) 
    #print "confusion_matrix: \n{}".format(cm) 
    plt.imshow(cm, interpolation='nearest', cmap=cmap) 
    plt.title(title) 
    plt.colorbar() 
    tick_marks =np.arange(len(fault_label))
    plt.xticks(tick_marks, fault_label.values(), rotation=45) 
    plt.yticks(tick_marks, fault_label.values()) 
    plt.tight_layout() 
    plt.ylabel('True label') 
    plt.xlabel('Predicted label') 
    plt.show()

def test_(): 
    y_true = np.load("data/y_true.npy") 
    y_pred = np.load("data/y_pred.npy") 
    print(classification_report(y_true, y_pred, 
          target_names = fault_label.values()))
    print(confusion_matrix(y_true, y_pred))
    print()
    plot_confusion_matrix(confusion_matrix(y_true, y_pred))

if __name__ == "__main__":
    test_()